In [2]:
import pandas as pd
import nltk
import re #regex mdule ekak
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
#nltk.download()

# Data set Reading


In [4]:
df = pd.read_csv('Resume Ranking Data Set.csv')
df_cp = df.copy()

In [5]:
df.head(20)

,accomplishments_segment,degree,education_segment,emails,job_titles,links,misc_segment,name,objectives_segment,phone,...,text,university_0,university_1,university_2,university_3,university_4,university_5,url,work_experience,work_segment
0,NaN,"MS , B.TECH , PhD",NaN,sutarvinayak2063@gmail.com,"full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...",NaN,Prasad Nagar,NaN,+919309161108,...,VINAYAK \nSUTAR \nLinkedIn \n+91 9309161108 ...,"national, mainpuri, uttar pradesh","college of arts, sundargarh, odisha","school of engineering, ernakulam, kerala",NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
1,NaN,"B.E , MS , M.TECH",M.Tech. Data Science\nJSS Science And Technolo...,pooj.vs@gmail.com,software engineer,pooj.vs,06/2017 – 07/2019\n07/2018 – 07/2019\nmailto:p...,Software Engineer,NaN,7411786822,...,POOJA V S\nSoftware Engineer\nRich exposure to...,"government, kandhamal, odisha","vidya vardhaka college of engineering, mysore,...",NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,Merchandising System' using PL/SQL.\nmaintain ...
2,Successfully cleared and completed the course ...,"MS , B.TECH , BS",Degree\nInstitute\nBoard/University\nYear\nPer...,khandelwal.ashwin5@gmail.com,NaN,https://www.linkedin.com/in/ashwin-khandelwal-...,NaN,Ashwin Khandelwal,"I seek challenging environment to work in, tha...",9462710290,...,Ashwin Khandelwal \n Email id – khand...,"narsee monjee institute of management studies,...",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,Organization / Institute\nDescription\nBharat ...
3,NaN,"B.E , MS , BS",NaN,NaN,"work from home , support analyst , software an...",NaN,NaN,Computer Engineering,NaN,919930485336,...,Assessment\nQ1. Why should you be hired for th...,"master, medak, telangana",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,1.0,NaN
4,NaN,"MS , B.TECH , BS",NaN,NaN,"work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,NaN, Python,NaN,918010295832,...,Assessment\nQ1. Why should you be hired for th...,"greater noida institute of technology, gautam ...",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
5,NaN,B.TECH,NaN,NaN,work from home,NaN,NaN,AKSHAY DESHBHRATAR,NaN,918888238622,...,Assessment\nQ1. Why should you be hired for th...,"government polytechnic, kachchh, gujarat",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,5.0,"TRAININGS\nThuse Electronics Private Limited,B..."
6,NaN,"B.E , MS , BS",NaN,NaN,"work from home , secretary , data scientist",NaN,NaN,kaashiv infotech,NaN,919884679445,...,Assessment\nQ1. Why should you be hired for th...,"jeppiaar engineering college, kancheepuram, ta...",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,POSIT IONS OF\nRESPONSIBILITY\nTRAININGS\n
7,NaN,"BTECH , B.TECH",NaN,NaN,"work from home , partner","in.programy.noq , https://github.com/kumaranki...",NaN,IIT Roorkee,NaN,917302202034,...,Assessment\nQ1. Why should you be hired for th...,"indian institute of technology, saharanpur, ut...","master, medak, telangana","institute of technology roorkee, hardwar, uttr...",NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
8,NaN,"B.TECH , BS",NaN,NaN,"project manager , work from home","https://play.google.com/store/apps/details , h...",NaN,West Bengal,NaN,918405978077,...,Assessment\nQ1. Why should you be hired for th...,"nshm knowledge campus, kolkata, west bengal",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
9,NaN,"MS , B.TECH , BS",NaN,NaN,"pizza delivery , work from home",https://github.com/ayush-09/Face_Mask_Detector...,NaN,Apache Cassandra,NaN,917060209421,...,Assessment\nQ1. Why should you be hired for th...,NaN,NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,TRAININGS\n


# Null Values

In [6]:
df.isnull().sum()#ek eka columns wala numm value count ek denwaaa

accomplishments_segment    1507
degree                       38
education_segment          1116
emails                     1509
job_titles                   27
links                       354
misc_segment               1551
name                          1
objectives_segment         1159
phone                       219
projects_segment            893
skills                        0
skills_segment             1105
text                          0
university_0               1007
university_1               1305
university_2               1480
university_3               1545
university_4               1568
university_5               1571
url                           0
work_experience             854
work_segment                893
dtype: int64

# Dropping the features which doesn't required

In [7]:
df.drop(['accomplishments_segment','education_segment','emails','misc_segment','name','objectives_segment','phone','projects_segment','skills_segment','text',
         'university_0','university_1','university_2','university_3','university_4','university_5','url','work_segment'],axis=1,inplace=True)
df.shape
     

(1573, 5)

# Total null values of features after dropping the features


In [9]:
print('After removing the columns from dataset total null values')
df.isnull().sum()

After removing the columns from dataset total null values


degree              38
job_titles          27
links              354
skills               0
work_experience    854
dtype: int64

# Handling Null values of features

In [10]:
df['degree'].fillna(df.degree.mode()[0],inplace=True)
df['links'].fillna('Missing',inplace=True)
df['work_experience'].fillna(0,inplace=True)
df['job_titles'].fillna('No Job',inplace=True)
df.head()


,degree,job_titles,links,skills,work_experience
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0


In [11]:
all_degrees = ''
for i in df.degree:
  if len(all_degrees) == 0:
    all_degrees = i
  else:
    all_degrees = all_degrees + ' , ' + i

all_degrees = all_degrees.split(',')
all_degrees = [re.sub("[\s.]","",i).upper() for i in all_degrees]
unique_degrees = set(all_degrees)
unique_degrees
     

{'BE', 'BS', 'BSC', 'BTECH', 'ME', 'MS', 'MSC', 'MTECH', 'PHD'}

In [12]:
df['bachelor_degrees'] = 'No Degree'
df['master_degrees'] = 'No Degree'
df['docterte_degrees'] = 'No Degree'
df['profiles'] = 'No Profile'
     

In [13]:
df.head()

,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,No Degree,No Degree,No Degree,No Profile
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0,No Degree,No Degree,No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,No Degree,No Degree,No Degree,No Profile
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,No Degree,No Degree,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,No Degree,No Degree,No Degree,No Profile


In [14]:
ind = 0
for i in df.degree:  
  lst = re.sub("[\s.]","",i).upper( )
  # print(lst)
  for j in lst.split(','):
    if j in ['BE', 'BS', 'BSC', 'BTECH']:
      if df.loc[ind,'bachelor_degrees'] == 'No Degree':
        df.loc[ind,'bachelor_degrees'] = j
      else:
        df.loc[ind,'bachelor_degrees'] = df.loc[ind,'bachelor_degrees'] + ' , '+ j
    elif j in ['ME', 'MS', 'MSC', 'MTECH']:      
      if df.loc[ind,'master_degrees'] == 'No Degree':
        df.loc[ind,'master_degrees'] = j
      else:
        df.loc[ind,'master_degrees'] = df.loc[ind,'master_degrees'] + ' , '+ j
    elif j == 'PHD':      
      if df.loc[ind,'docterte_degrees'] == 'No Degree':
        df.loc[ind,'docterte_degrees'] = j
      else:
        df.loc[ind,'docterte_degrees'] = df.loc[ind,'docterte_degrees'] + ' , '+ j
  ind +=1

In [15]:
df

,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,BTECH,MS,PHD,No Profile
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0,BE,"MS , MTECH",No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,"BTECH , BS",MS,No Degree,No Profile
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,"BE , BS",MS,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,"BTECH , BS",MS,No Degree,No Profile
...,...,...,...,...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0,BE,"MS , MTECH",No Degree,No Profile
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0,BS,MS,No Degree,No Profile
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,MS,classifier,Missing,"natural language , matplotlib , boost , custom...",0.0,No Degree,MS,No Degree,No Profile


In [18]:
ind = 0
for i in df.links:
  lst = re.sub("[\s]","",i)
  # print(lst)
  for j in lst.split(','):
    if j.find('github') != -1:          
      df.loc[ind,'profiles'] = 'Github'
    elif j.find('linkedin') != -1:
      if df.loc[ind,'profiles'] == 'No Profile':
        df.loc[ind,'profiles'] = 'Linkedin'
      else:
        df.loc[ind,'profiles'] = df.loc[ind,'profiles'] + ' , ' + 'Linkedin'
  ind +=1

In [17]:
df

,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,BTECH,MS,PHD,Github
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0,BE,"MS , MTECH",No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,"BTECH , BS",MS,No Degree,Linkedin
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,"BE , BS",MS,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,"BTECH , BS",MS,No Degree,Github
...,...,...,...,...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0,BE,"MS , MTECH",No Degree,"Github , Linkedin"
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0,BS,MS,No Degree,No Profile
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,MS,classifier,Missing,"natural language , matplotlib , boost , custom...",0.0,No Degree,MS,No Degree,No Profile


In [19]:
# list(df['work_experience'].unique())
df.work_experience.value_counts()

 0.0       854
 4.0       151
 2.0        91
 1.0        87
 3.0        81
 5.0        73
 6.0        52
 8.0        34
 7.0        32
 9.0        23
 14.0       14
 11.0       12
 10.0       12
 12.0       11
 13.0        6
 18.0        5
 15.0        4
 17.0        4
 21.0        4
 22.0        2
 2005.0      2
 2004.0      2
 16.0        2
 2003.0      2
 2006.0      2
 20.0        2
 2002.0      1
 2010.0      1
 2001.0      1
 2012.0      1
-3.0         1
 1999.0      1
 2013.0      1
 2008.0      1
 19.0        1
Name: work_experience, dtype: int64

In [26]:
ind = 0
for i in df.work_experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['work_experience'] = df['work_experience'].astype(int)  

In [27]:
df

,job_titles,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"full stack developer , student","algorithms , interview , visualisation , robot...",0,BTECH,MS,PHD,Github
1,software engineer,"core java , algorithms , sql tools , deep lear...",0,BE,"MS , MTECH",No Degree,No Profile
2,No Job,"digital signal processing , algorithms , capac...",0,"BTECH , BS",MS,No Degree,"Linkedin , Linkedin"
3,"work from home , support analyst , software an...","big data analytics , award , training , comput...",1,"BE , BS",MS,No Degree,No Profile
4,"work from home , classifier","functionality , algorithms , spark , data visu...",0,"BTECH , BS",MS,No Degree,Github
...,...,...,...,...,...,...,...
1568,"branch manager , embedded software developer ,...","digital communication , api , atom , robot , c...",2,BE,"MS , MTECH",No Degree,"Github , Linkedin"
1569,"devops , senior android developer , team lead","reporting , functionality , socket.io , mockit...",2,BS,MS,No Degree,No Profile
1570,"devops engineer , traffic manager , release ma...","version control , renewals , nfs , telugu , vi...",12,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,classifier,"natural language , matplotlib , boost , custom...",0,No Degree,MS,No Degree,No Profile


In [41]:
df.drop(['degree','links'],axis=1,inplace=True)
     

KeyError: "['degree', 'links'] not found in axis"

In [29]:
df.isnull().sum()


job_titles          0
skills              0
work_experience     0
bachelor_degrees    0
master_degrees      0
docterte_degrees    0
profiles            0
dtype: int64

In [30]:
df

,job_titles,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"full stack developer , student","algorithms , interview , visualisation , robot...",0,BTECH,MS,PHD,Github
1,software engineer,"core java , algorithms , sql tools , deep lear...",0,BE,"MS , MTECH",No Degree,No Profile
2,No Job,"digital signal processing , algorithms , capac...",0,"BTECH , BS",MS,No Degree,"Linkedin , Linkedin"
3,"work from home , support analyst , software an...","big data analytics , award , training , comput...",1,"BE , BS",MS,No Degree,No Profile
4,"work from home , classifier","functionality , algorithms , spark , data visu...",0,"BTECH , BS",MS,No Degree,Github
...,...,...,...,...,...,...,...
1568,"branch manager , embedded software developer ,...","digital communication , api , atom , robot , c...",2,BE,"MS , MTECH",No Degree,"Github , Linkedin"
1569,"devops , senior android developer , team lead","reporting , functionality , socket.io , mockit...",2,BS,MS,No Degree,No Profile
1570,"devops engineer , traffic manager , release ma...","version control , renewals , nfs , telugu , vi...",12,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,classifier,"natural language , matplotlib , boost , custom...",0,No Degree,MS,No Degree,No Profile


In [31]:
lemmatizer = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))        

In [32]:
def get_wordnet_pos(word):    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
     

In [33]:
def get_tf_idf_cosine_similarity(job_desc,all_resumes):    
    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    tf_idf_desc_vector = tf_idf_vect.fit_transform([job_desc]).todense()
    tf_idf_resume_vector = tf_idf_vect.transform(all_resumes).todense()
    cosine_similarity_list = []
    for i in range(len(tf_idf_resume_vector)):
        cosine_similarity_list.append(cosine_similarity(tf_idf_desc_vector,tf_idf_resume_vector[i])[0][0])
    return cosine_similarity_list    

# Opening the Job Description from the text file

In [38]:
with open('Job Description.txt','r',encoding = 'utf-8') as f:
  file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]

FileNotFoundError: [Errno 2] No such file or directory: 'Job Description.txt'

In [40]:
job_description = ''

for i in file_desc_lst:
  if len(job_description) == 0:
    job_description = str(i)
  else:
    job_description = job_description + ' ' + str(i)

NameError: name 'file_desc_lst' is not defined

In [39]:
job_description

NameError: name 'job_description' is not defined

# Reading all resume's from the dataset and convert into text which will appended into a list


In [79]:
all_resume_text = []

for i in df.iloc[:].values:
  s = ''
  for j in list(i):
    if len(s) == 0:
      s = str(j)
    else:
      s = s + ' , ' + str(j)
  all_resume_text.append(s)

In [80]:
all_resume_text

['full stack developer , student , algorithms , interview , visualisation , robot , deep learning , sqlite , sen , cracking , information technology , hindi , maintenance , english , cpp , mongodb , linkedin , adt , iit , gaming , development work , persona , data analysis , knowledge sharing , django , research , flask , app , web , officers , 0 , BTECH , MS , PHD , No Profile',
 'software engineer , core java , algorithms , sql tools , deep learning , media analysis , nlp , learning techniques , chess , analytics , twitter , social media , project , grasp , puzzles , application , programming languages , pl/sql , facebook , information science , rest , government , machine learning , accessibility , fusion charts , sentiment analysis , big data , kannada , oracle retail , merchandising , data science , python , javascript , eclipse , jdeveloper , web , layout , technology , information retrieval , drawing , hobbies , html , 0 , BE , MS , MTECH , No Degree , No Profile',
 'No Job , di

In [81]:
cos_sim_list = get_tf_idf_cosine_similarity(job_description,all_resume_text)

C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-

C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\maduu\anaconda3\lib\site-

In [82]:
zipped_resume_rating = zip(cos_sim_list,df_cp.name,[x for x in range(len(df))])
sorted_resume_rating_list = sorted(zipped_resume_rating, key = lambda x: x[0], reverse=True)
     



# Generating the DataFrame which contains the name of the candidate with his/her cosine score(%)

In [83]:
resume_score = [round(x*100,2) for x in cos_sim_list]

In [84]:
resume_score

[25.42,
 32.91,
 29.66,
 38.7,
 35.09,
 19.89,
 33.38,
 34.28,
 29.77,
 31.28,
 27.75,
 23.46,
 24.88,
 35.36,
 29.77,
 29.77,
 36.78,
 32.1,
 32.17,
 29.05,
 37.22,
 33.15,
 11.48,
 20.3,
 21.4,
 33.49,
 32.48,
 29.67,
 32.48,
 24.88,
 39.99,
 31.71,
 37.21,
 31.13,
 23.2,
 31.71,
 27.74,
 36.11,
 28.13,
 32.74,
 18.04,
 34.25,
 32.82,
 32.82,
 44.02,
 35.71,
 24.31,
 29.67,
 35.94,
 20.58,
 33.38,
 14.36,
 35.1,
 34.24,
 36.36,
 22.97,
 20.58,
 27.84,
 10.89,
 25.3,
 35.45,
 21.88,
 33.83,
 33.83,
 27.84,
 44.18,
 35.09,
 29.77,
 37.51,
 35.36,
 34.0,
 31.01,
 32.04,
 31.67,
 33.38,
 47.37,
 37.69,
 34.53,
 32.04,
 25.71,
 23.2,
 32.48,
 22.97,
 31.13,
 38.36,
 42.68,
 43.31,
 29.28,
 33.03,
 34.28,
 38.67,
 32.68,
 24.88,
 41.94,
 35.16,
 36.75,
 32.64,
 39.37,
 50.84,
 37.88,
 34.38,
 32.52,
 30.39,
 40.19,
 29.96,
 31.53,
 30.03,
 23.2,
 29.67,
 22.01,
 26.16,
 21.05,
 30.69,
 33.55,
 38.28,
 10.83,
 34.24,
 26.79,
 20.3,
 36.8,
 25.71,
 33.15,
 33.15,
 33.67,
 38.92,
 34.5,
 40.7

,name,resume_score(%)
464,Rucha Patwardhan,59.38
1542,Technical Architect,51.72
1095,Cloud Infrastructure,51.66
98,Microsoft Word,50.84
75,Credit Card,47.37
